# Задание 1 готово

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [18]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [36]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3]) == foldl2(lambda x, y: x + y, 0, [1, 2, 3])

True

In [35]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3]) == foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

True

In [34]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]) == foldl2(f, 1, [1, 2, 3]), foldr(f, 1, [1, 2, 3]) == foldr2(f, 1, [1, 2, 3])

(True, True)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [22]:
def foldl2(f, x0, lst):
    return foldr(lambda  x,y: lambda z: y(f(z,x)) , lambda x: x, lst)(x0)

foldl2(f, 1, [1,2,3])
#test
foldl2(lambda x, y: x / y, 8, [2, 4, 8]) == foldl(lambda x, y: x / y, 8, [2, 4, 8])

True

In [61]:
def foldr2(f, x0, lst):
    return foldl(lambda x,y: lambda z: x(f(y, z)), lambda x: x, lst)(x0)

#тесты

#сумма
print(foldr(lambda x, y: x + y, 0, [1, 2, 3]) == foldr2(lambda x, y: x + y, 0, [1, 2, 3]))

#конкатенация
print(foldr(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3]) == foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3]))

#деление
f = lambda x, y: x / y
print(foldr(f, 1, [1, 2, 3]) == foldr2(f, 1, [1, 2, 3]))

#то, на чём у меня всё завалилось
f = lambda x, y: '(%s, %s)' % (x, y)
print(foldr(f, 1, [1, 2, 3]) == foldr2(f, 1, [1, 2, 3]))



True
True
True
True


# Задание 2 готово

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [96]:
def check_inv(a, b):
    b_dict = dict()
    b_dict_zeros = dict()
    for letter in b:
        if letter in b_dict:
            b_dict[letter] = b_dict[letter] + 1
        else:
            b_dict[letter] = 1
            b_dict_zeros[letter] = 0
    for letter_number in range(len(a)-len(b)+1):
        if a[letter_number] in b_dict:
            b_dict_temp = b_dict.copy()
            b_dict_temp[a[letter_number]] = b_dict_temp[a[letter_number]] - 1
            for letter_number_temp in range(letter_number+1, letter_number+len(b)):         
                if a[letter_number_temp] in b_dict_temp:
                    if b_dict_temp[a[letter_number_temp]] > 0:
                        b_dict_temp[a[letter_number_temp]] = b_dict_temp[a[letter_number_temp]] - 1                        
                        if b_dict_temp == b_dict_zeros:
                            return True
                        continue
                break
    return False

In [97]:
check_inv('abcrotm','tro'), check_inv('good', 'god'), check_inv('caaab', 'baa')

(True, False, True)

 Сложность: O(nm)

# Задание 3 готово

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [32]:
class Tree:        
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left:
            yield from iter(self.left)
        if self.right:
            yield from iter(self.right)
        else:
            yield self.value
    
    def __str__(self):
        a = ''
        b = ''
        if self.left:
            a = str(self.left)
        if self.right:
            b = str(self.right)
        return a + b + ' ' + str(self.value)
        
    def __repr__(self):
        return str(self)
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

repr(tree)

' 3 4 1 2 0'

# Задание 4 готово

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

In [176]:
def isInt(syms):
    try:
        int(syms)
        return True
    except:
        return False

def toReversePolishNotation(expr):
    expr = expr.replace('(', '( ').replace(')', ' )')
    queue = expr.split(' ')
    res = ''
    stack = []
    for sym in queue:
        if isInt(sym):
            res = res + sym + ' '
        elif sym == '+' or sym == '-':
            while stack and (stack[-1] == '*' or stack[-1] == '\\'):
                res = res + stack.pop() + ' '
            stack.append(sym)
        elif sym == '*' or sym == '\\':
            stack.append(sym)                
        elif sym == '(':
            stack.append(sym)
        elif sym == ')':
            while stack[-1] != '(':
                res = res + stack.pop() + ' '
            stack.pop()
    while stack:
        res = res + stack.pop() + ' '
    return res[:-1]
                
        
def calc(expr):
    rpn = toReversePolishNotation(expr)
    syms = rpn.split(' ')
    i = 0
    while len(syms)>1:
        if isInt(syms[i]):
            i = i + 1
        else:
            if syms[i] == '+':
                syms[i-2] = int(syms[i-2]) + int(syms[i-1])
            if syms[i] == '-':
                syms[i-2] = int(syms[i-2]) - int(syms[i-1])
            if syms[i] == '*':
                syms[i-2] = int(syms[i-2]) * int(syms[i-1])
            if syms[i] == '\\':
                syms[i-2] = int(syms[i-2]) / int(syms[i-1])            
            del syms[i-1]
            del syms[i-1]
            i = i - 1
    return syms[-1]

(calc('2 * (15 - 3 * 4) - 2') == 4, 
calc('(1 + 2) * 4 + 3') == 15,
calc('(4 + 4) \ 4') == 2,
calc('4 + 4 \ 4') == 5)

(True, True, True, True)